Before you begin, make sure you are connected.

Installing some necessary packages.
!pip install gap-stat
!pip install joblib
!pip install PyPDF2

In [8]:
import os
import glob
import json
import PyPDF2
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import gap_statistic as gs
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Prepocessing functions

In [9]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.
    """
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
    return text

Create a corpus (in this case of the international agreements.
Determine the absolute path to the folder with PDF files.
This goes one level up from the current script folder and then to 'data'

In [11]:
#data_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "UNFCCC_Data\\international_agreements"))
data_dir = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "..", "UNFCCC_Data\\international_agreements"))
pdf_files = glob.glob(os.path.join(data_dir, "*.pdf"))

In [14]:
def clean_text(text):
  """
  Cleans a text document by removing stop words, converting to lowercase,
  removing punctuation, and normalizing symbols.
  """
  stopwords = nltk.corpus.stopwords.words('english')  # Download stopwords if needed
  text = text.lower()
  text = "".join([c for c in text if c not in string.punctuation])  # Remove punctuation
  text = re.sub(r'[^\w\s]', '', text)  # Normalize symbols
  text = " ".join([word for word in text.split() if word not in stopwords])  # Remove stopwords
  return text

In [15]:
def tokenize_text(text):
  """
  Tokenizes a text document into individual words.
  """
  return word_tokenize(text)

In [16]:
def normalize_text(text, method='stemming'):
  """
  Normalizes a text document using stemming or lemmatization.
  """
  if method == 'stemming':
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text])
  elif method == 'lemmatization':
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text])
  else:
    raise ValueError("Invalid normalization method.")

In [ ]:
from sentence_transformers import SentenceTransformer
from faiss import IndexFlatL2

In [ ]:
# Function to choose between CPU or GPU mode
def choose_mode():
  while True:
    mode = input("Choose mode (CPU or GPU): ").upper()
    if mode in ("CPU", "GPU"):
      return mode
    else:
      print("Invalid choice. Please enter CPU or GPU.")

In [ ]:
# Load the pre-trained sentence transformer model (replace with your chosen model)
model = SentenceTransformer('all-mpnet-base-v2')

# Extract patent abstracts and identifiers
abstract_texts = data[~data['abstract'].isna()]['abstract']
patent_ids = data[~data['abstract'].isna()]['lens_id']

# Encode the patent abstracts using the sentence transformer model
abstract_embeddings = model.encode(abstract_texts.tolist())

# User-selected mode (CPU or GPU)
mode = choose_mode()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Define a function to compute cosine similarity between a query and all patents
def get_cosine_similarities(query_embedding):
  # Search for nearest neighbors (all patents in this case)
  distances, neighbors = index.search(query_embedding.reshape(1, -1), len(abstract_embeddings))
  # Convert distances to similarities (lower distance, higher similarity)
  similarities = 1 - distances.squeeze()
  # Create a DataFrame with patent IDs, similarities, and exclude the query itself
  return pd.DataFrame({'patent_id': patent_ids.iloc[neighbors].tolist(), 'similarity': similarities.tolist()})[1:]  # Exclude query (index 0)

In [ ]:
# Generate edgelist with cosine similarities for each country
edgelist = []
for i, embedding in enumerate(abstract_embeddings):
  similarities_df = get_cosine_similarities(embedding)
  similarities_df['source_patent_id'] = patent_ids.iloc[i]
  edgelist.append(similarities_df)

# Combine edge DataFrames into a single DataFrame
edgelist_df = pd.concat(edgelist, ignore_index=True)

# You can now use the edgelist_df for further analysis (e.g., filtering by similarity threshold)
print(edgelist_df)